In [ ]:
import respiratory_extraction.dataset as repository

dataset = repository.from_default()

subject = 'Proband16'
scenario = '101_natural_lighting'

subject_frames, params = dataset.read_video_gray(subject, scenario)

In [ ]:
hyperparameters = {
    'OFP_maxCorners': 100,
    'OFP_qualityLevel': 0.1,
    'OFP_minDistance': 7,
    'OFP_mask': None,
    'OFP_QualityLevelRV': 0.05,
    'OFP_winSize': (15, 15),
    'OFP_maxLevel': 2,
    'FSS_maxCorners': 100,
    'FSS_qualityLevel': 0.1,
    'FSS_minDistance': 7,
    'FSS_mask': None,
    'FSS_QualityLevelRV': 0.05,
    'FSS_FPN': 5,
    'Filter_order': 3,
    'Filter_LowPass': 0.1,
    'Filter_HighPass': 0.6,
    'RR_Algorithm_PC_Height': None,
    'RR_Algorithm_PC_Threshold': None,
    'RR_Algorithm_PC_MaxRR': 45,
    'RR_Algorithm_NFCP_qualityLevel': 0.6
}

In [ ]:
import numpy as np
import respiratory_extraction.utils as utils
import respiratory_extraction.models.optical_flow as optical_flow


def find_roi(frame: np.ndarray) -> tuple[int, int, int, int]:
    """
    Find the region of interest (ROI) based on the face detection
    :param frame: The frame to find the ROI
    :return: The region of interest (ROI) coordinates (x, y, w, h)
    """

    faces = utils.detect_faces(frame)

    if len(faces) == 0:
        raise ValueError('No face detected in the first frame')
    elif len(faces) > 1:
        raise ValueError('Multiple faces detected in the first frame')

    # First face position and size
    x, y, w, h = faces[0]

    scale_x = int(w * 0.2)

    # Calculate the region of interest (ROI) based on the face
    chest_x = x - scale_x
    chest_y = int(y + h + h * 0.7)
    chest_w = w + scale_x * 2
    chest_h = int(h * 0.5)

    return chest_x, chest_y, chest_w, chest_h

In [ ]:
from enum import Enum


class FeaturePointStrategy(Enum):
    special = 'special'
    default = 'default'
    roi = 'roi'
    roi_special = 'roi_special'

    def __str__(self):
        return self.value

    def uses_roi(self) -> bool:
        return self == FeaturePointStrategy.roi or self == FeaturePointStrategy.roi_special

    def uses_special(self) -> bool:
        return self == FeaturePointStrategy.special or self == FeaturePointStrategy.roi_special


def get_feature_points(
        frame: np.ndarray,
        quality_level=float(0.3),
        quality_level_rv=float(0.05),
        strategy=FeaturePointStrategy.default,
) -> np.ndarray:
    roi_mask = None
    if strategy.uses_roi():
        roi = find_roi(frame)
        roi_mask = utils.roi_to_mask(frame, roi)

    if strategy.uses_special():
        feature_points = optical_flow.special_feature_point_selection(
            frame,
            fpn=hyperparameters['FSS_FPN'],
            mask=roi_mask,
            quality_level=quality_level,
            quality_level_rv=quality_level_rv,
        )
    else:
        feature_points = optical_flow.feature_point_selection(
            frame,
            mask=roi_mask,
            quality_level=quality_level,
            quality_level_rv=quality_level_rv,
        )

    return feature_points

In [ ]:
import cv2


def extract_feature_point_movement(frames: np.ndarray, feature_points: np.ndarray) -> np.ndarray:
    lk_params = {
        'winSize': hyperparameters['OFP_winSize'],
        'maxLevel': hyperparameters['OFP_maxLevel'],
    }
    total_frame = len(frames)

    # Store the feature points for each frame
    feature_point_matrix = np.zeros((int(total_frame), feature_points.shape[0], 2))

    # Store the feature points for the first frame
    feature_point_matrix[0, :, 0] = feature_points[:, 0, 0].T
    feature_point_matrix[0, :, 1] = feature_points[:, 0, 1].T

    # Calculate the optical flow of the feature points for each frame
    for inx in range(1, total_frame):
        current_frame = frames[inx - 1]
        next_frame = frames[inx]

        new_positions, _, _ = cv2.calcOpticalFlowPyrLK(
            current_frame,
            next_frame,
            feature_points,
            None,
            **lk_params)

        feature_points = new_positions.reshape(-1, 1, 2)
        feature_point_matrix[inx, :, 0] = feature_points[:, 0, 0].T
        feature_point_matrix[inx, :, 1] = feature_points[:, 0, 1].T

    return feature_point_matrix

In [ ]:
def extract_respiratory_signal(
        frames: np.ndarray,
        fps: int,
        quality_level=float(0.3),
        strategy=FeaturePointStrategy.default,
        use_cgof=False,
        use_filter=True,
        use_normalization=False,
) -> np.ndarray:
    feature_points = get_feature_points(
        frames[0],
        strategy=strategy,
        quality_level=quality_level)

    # Extract the movement of the feature points for each frame
    feature_point_movements = extract_feature_point_movement(frames, feature_points)

    # Calculate the amplitude of the feature points for each frame
    point_amplitudes = np.sqrt(feature_point_movements[:, :, 0] ** 2 + feature_point_movements[:, :, 1] ** 2)

    # Calculate the amplitude of the feature points for each frame
    respiratory_signal = np.sum(point_amplitudes, 1) / point_amplitudes.shape[1]

    # Correlation-Guided Optical Flow Method
    if use_cgof:
        respiratory_signal = optical_flow.correlation_guided_optical_flow_method(point_amplitudes, respiratory_signal)

    # Butterworth Filter
    if use_filter:
        respiratory_signal = optical_flow.butterworth_filter(
            respiratory_signal,
            fps,
            lowpass=hyperparameters['Filter_LowPass'],
            highpass=hyperparameters['Filter_HighPass'],
        )

    # Normalization
    if use_normalization:
        respiratory_signal = optical_flow.normalize_signal(respiratory_signal)

    return respiratory_signal

In [ ]:
breathing_signal_roi = extract_respiratory_signal(
    subject_frames,
    params.fps,
    quality_level=hyperparameters['OFP_qualityLevel'],
    strategy=FeaturePointStrategy.roi,
    use_cgof=True,
    use_filter=True,
    use_normalization=True,
)

In [ ]:
import plotly.express as px

fig = px.line(y=breathing_signal_roi, title='Respiratory Signal')
fig.show()

In [ ]:
get_ground_truth_rr = dataset.get_ground_truth_rr(subject, scenario)
get_ground_truth_rr, get_ground_truth_rr * 60

In [ ]:
import pandas as pd

from tqdm.auto import tqdm

evaluation_results = {}

for fp_strategy in tqdm(FeaturePointStrategy):
    breathing_signal = extract_respiratory_signal(
        subject_frames,
        params.fps,
        quality_level=hyperparameters['OFP_qualityLevel'],
        strategy=fp_strategy,
        use_cgof=True,
        use_filter=True,
        use_normalization=True,
    )

    extraction = optical_flow.FrequencyExtraction(breathing_signal, params.fps)

    evaluation_results[fp_strategy] = {
        'FFT ': extraction.fft(),
        'PC': extraction.peak_counting(),
        'CP': extraction.crossing_point(),
        'NFCP': extraction.negative_feedback_crossover_point_method()
    }

In [ ]:
evaluation_rows = []

for fp_strategy, methods in evaluation_results.items():
    for method, value in methods.items():
        evaluation_rows.append({
            'fp_strategy': fp_strategy.value,
            'method': method,
            'respiration_frequency': value,
            'respiration_rate': value * 60,
            'error': abs(get_ground_truth_rr * 60 - value * 60),
        })

evaluation = pd.DataFrame(evaluation_rows)
evaluation

In [ ]:
import matplotlib.pyplot as plt

# Plot the error for each Strategy and Method
fig, axs = plt.subplots(1, len(FeaturePointStrategy), figsize=(15, 5))

for idx, fp_strategy in enumerate(FeaturePointStrategy):
    strategy_evaluation = evaluation[evaluation['fp_strategy'] == fp_strategy.value]

    axs[idx].bar(strategy_evaluation['method'], strategy_evaluation['error'])
    axs[idx].set_title(f'{fp_strategy}')
    axs[idx].set_ylabel('Error (BPM)')

plt.show()

In [ ]:
old_gray = subject_frames[0]

default_points = optical_flow.feature_point_selection(old_gray, mask=None)
special_points = optical_flow.special_feature_point_selection(old_gray, mask=None)

subject_roi = find_roi(old_gray)
# subject_roi_mask = utils.roi_to_mask(old_gray, subject_roi)
# 
# roi_points = optical_flow.feature_point_selection(old_gray, mask=subject_roi_mask)
# special_roi = optical_flow.special_feature_point_selection(old_gray, mask=subject_roi_mask)

print(f'Feature Points: {default_points.shape[0]}')
print(f'Special Points: {special_points.shape[0]}')
# print(f'ROI Points: {roi_points.shape[0]}')
# print(f'Special ROI Points: {special_roi.shape[0]}')

# Plot the first frame with the feature points
plt.imshow(old_gray, cmap='gray')

# Draw the region of interest (ROI)
roi_x, roi_y, roi_w, roi_h = subject_roi
plt.gca().add_patch(plt.Rectangle(
    (roi_x, roi_y), roi_w, roi_h,
    linewidth=1, edgecolor='r', facecolor='none'))

for iny in range(default_points.shape[0]):
    plt.scatter(default_points[iny, 0, 0],
                default_points[iny, 0, 1],
                c='r', s=2.5)

for iny in range(special_points.shape[0]):
    plt.scatter(special_points[iny, 0, 0],
                special_points[iny, 0, 1],
                c='b', s=2.5)

# for iny in range(roi_points.shape[0]):
#     plt.scatter(roi_points[iny, 0, 0],
#                 roi_points[iny, 0, 1],
#                 c='#FFFF00', s=2.5)
# 
# for iny in range(special_roi.shape[0]):
#     plt.scatter(special_roi[iny, 0, 0],
#                 special_roi[iny, 0, 1],
#                 c='#FF00FF', s=2.5)

plt.show()